In [1]:
import numpy as np

from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.csgraph import csgraph_from_dense

# Типы и Структуры Данных. ЛР4
## Разреженные матрицы

Необходимо реализовать либо КРМ, либо РСФ: упаковку, распаковку, сложение двух матриц и произведение двух матриц.

В лабораторной не допускается сплошное извлечение элементов матриц (вида "извлечь элемент с координатами [i][j] из упакованной матрицы"), требуется использовать алгоритмы просмотра массива ненулевых элементов.

In [2]:
A = np.array([
    [5, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [1, 0, 4, 0, 0, 4, 0, 0, 0],
    [1, 0, 3, 0, 0, 0, 7, 0, 0],
    [2, 7, 1, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 2, 0, 0, 0, 0, 2],
])

In [3]:
B = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 2, 8, 2, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [1, 0, 4, 0, 0, 2, 0, 0, 0],
    [1, 0, 2, 0, 0, 0, 1, 0, 0],
    [2, 9, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 2, 0, 0, 0, 0, 3],
])

In [4]:
count = 0
  
rows = len(A)
cols = len(A[0])
  
size = rows * cols
   
for i in range(0, rows):  
    for j in range(0, cols):  
        if(A[i][j] == 0):  
            count = count + 1

if(count > (size/2)):  
    print("Данная матрица разреженная")
else:  
    print("Данная матрица плотная")

Данная матрица разреженная


In [5]:
# калькуляция разреженности матрицы A
sparsity = 1.0 - np.count_nonzero(A) / A.size
print(sparsity)

0.7654320987654322


In [6]:
# функция упаковки матрицы
def get_csr_representation(matrix):
    rows_index = []
    columns = []
    elements = []
    row_len = len(matrix)
    col_len = len(matrix[0])
    for i in range(row_len):
        rows_index.append(len(columns))
        for j in range(col_len):
            if(matrix[i][j] != 0):
                columns.append(j)
                elements.append(matrix[i][j])
    rows_index.append(len(columns))
    
    return rows_index, columns, elements

In [7]:
# функция распаковки матрицы
def get_dense_representation(csr_matrix):
    size = (len(csr_matrix[0])-1, len(csr_matrix[0])-1)
    dense_matrix = np.zeros(size)
    for i in range(size[0]):
        for j in range(csr_matrix[0][i], csr_matrix[0][i+1]):
            if j >= len(csr_matrix[2]):
                continue
            dense_matrix[i][csr_matrix[1][j]] = csr_matrix[2][j]
            
    return dense_matrix    

In [8]:
A

array([[5, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 4, 0, 0, 0, 0],
       [1, 0, 4, 0, 0, 4, 0, 0, 0],
       [1, 0, 3, 0, 0, 0, 7, 0, 0],
       [2, 7, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 2]])

In [9]:
B

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 2, 8, 2, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 4, 0, 0, 0, 0],
       [1, 0, 4, 0, 0, 2, 0, 0, 0],
       [1, 0, 2, 0, 0, 0, 1, 0, 0],
       [2, 9, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 3]])

In [10]:
A*B

array([[ 5,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  2,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  9,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 16,  0,  0,  0,  0],
       [ 1,  0, 16,  0,  0,  8,  0,  0,  0],
       [ 1,  0,  6,  0,  0,  0,  7,  0,  0],
       [ 4, 63,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  4,  0,  0,  0,  0,  6]])

In [11]:
# print('Упакованная CSR матрица A:')
# A_csr

In [12]:
# print('Упакованная CSR матрица B:')
# B_csr

get_csr_representation(A*B)

In [13]:
get_csr_representation(A*B)

([0, 1, 2, 4, 5, 6, 9, 12, 15, 17],
 [0, 1, 0, 2, 3, 4, 0, 2, 5, 0, 2, 6, 0, 1, 7, 3, 8],
 [5, 2, 1, 2, 9, 16, 1, 16, 8, 1, 6, 7, 4, 63, 1, 4, 6])

In [15]:
# функция умножения упакованных матриц
def sparse_csr_multiplication(matrix_A, matrix_B):
    if matrix_A == matrix_B:
        elements = [i*i for i in matrix_A[2]]
        res_matr = (matrix_A[0], matrix_A[1], elements)
        return res_matr
    
    rows_A, columns_A, elements_A = matrix_A
    rows_B, columns_B, elements_B = matrix_B
    result = ([], [], [])
    size = len(matrix_A[0])-1
    for i in range(size):
    
    
#     rows, columns, elements = matrix_A
#     rows_count = len(matrix_B)
#     col_count = len(matrix_B[0])
#     C = [[0 for _ in range(col_count)] for _ in range(rows_count)]
#     print(C)
#     for col in range(col_count):
#         for row in range(rows_count):
#             for j in range(rows[row],rows[row+1]):
#                 C[row][col] += elements[j] * matrix_B[columns[j]][col]
    
#     return C


sparse_csr_multiplication(A_csr, B_csr)

IndentationError: expected an indented block (<ipython-input-15-fa12482cbe37>, line 28)

In [ ]:
# функция сложения упакованных матриц
def sparse_csr_add(matrix_A, matrix_B):
    pass

In [ ]:
print('Исходная матрица B:')
print(B)

In [ ]:
print('Упакованная CSR матрица A:')
A_csr = get_csr_representation(A)
A_csr

In [ ]:
print('Упакованная CSR матрица B:')
B_csr = get_csr_representation(B)
B_csr

In [ ]:
print('Распаковка, матрица A:')
get_dense_representation(A_csr)

In [ ]:
print('Распаковка, матрица B:')
get_dense_representation(B_csr)

In [ ]:
get_csr_representation(A*A)

In [ ]:
print('Умножение A*A:')
sparse_csr_multiplication(A_csr, A_csr)

In [ ]:
print('Умножение A*B:')
sparse_csr_multiplication(A_csr, B_csr)

In [ ]:
print('Сложение:')

## Проверка
Для проверки результатов используем метод [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) (Compressed Sparse Row matrix) модуля [Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) из библиотеки [SciPy](https://docs.scipy.org/doc/scipy/reference/index.html)

In [ ]:
# convert to sparse matrix (Compressed Sparse Row - CSR method)
print('Упакованная CSR матрица:')
S = csr_matrix(A)
print(S)